**BOOK RECOMMENDATION SYSTEM - CONTENT BASED FILTERING**

EDA with Python Programming

In [2]:
!pip install seaborn
#hi

     ------------------------------------ 293.3/293.3 kB 348.1 kB/s eta 0:00:00
     -------------------------------------- 14.8/14.8 MB 263.7 kB/s eta 0:00:00
     -------------------------------------- 11.2/11.2 MB 765.5 kB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 743.1 kB/s eta 0:00:00
     -------------------------------------- 163.0/163.0 kB 1.4 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ---------------------------------------- 1.0/1.0 MB 926.6 kB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 48.9/48.9 kB 822.7 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 838.2 kB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     -------------------------------------- 502.3/502.3 kB 1.2 MB/s eta 0:00:00
     -----------------

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\f2py.exe' -> 'C:\\Python311\\Scripts\\f2py.exe.deleteme'


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd


df = pd.read_csv('books.csv')
print(f"In this dataframe, there are:\n{df.shape[0]} rows,\n{df.shape[1]} columns")

print("\n","The attributes/features are:")

df.dtypes

In this dataframe, there are:
6810 rows,
12 columns

 The attributes/features are:


isbn13              int64
isbn10             object
title              object
subtitle           object
authors            object
categories         object
thumbnail          object
description        object
published_year    float64
average_rating    float64
num_pages         float64
ratings_count     float64
dtype: object

Rows: 6810 
Attributes: 12 

**Analysis of Missing Data**

In [9]:
# print(df.iloc[51])
# df.drop(51,inplace=True)
# print(df.iloc[51])

In [2]:
percent_missing = df.isnull().sum() * 100 / len(df)
df.isnull().sum()
print(round(percent_missing,2))
# Therefore we can conclude that subtitle column has incomplete data

isbn13             0.00
isbn10             0.00
title              0.00
subtitle          65.04
authors            1.06
categories         1.45
thumbnail          4.83
description        3.85
published_year     0.09
average_rating     0.63
num_pages          0.63
ratings_count      0.63
dtype: float64


**Dimensionality reduction**
1. There are 65.02% of missing values in the feature 'subtitle', so we can conclude it is as incomplete data and drop the feature

2. As the 'thumbnail','published_year', 'num_pages' attributes are irrelevent to our analysis, we have dropped them.

3. Also, the features 'isbn10' and 'isbn13' serve the same purpose of unique identification of the book titles, we can consider the title for the identification and drop the 'isbn10' and 'isbn13' attributes

In [3]:
#Deletion of Unneccessary columns
df.columns

df.drop(columns=['subtitle','thumbnail','isbn10','isbn13','num_pages','published_year'],inplace=True)

#After Deleting, the columns are
df.columns

Index(['title', 'authors', 'categories', 'description', 'average_rating',
       'ratings_count'],
      dtype='object')

**Handling missing data**
We drop the null instances for the columns: 'authors', 'categories','description'

Perform mean value imputation for:
'average_rating','ratings_count'

In [4]:
df['ratings_count'].fillna(df['ratings_count'].mean(), inplace=True)
df['average_rating'].fillna(df['average_rating'].mean(), inplace=True)

In [5]:
df = df.dropna(subset=['authors', 'categories','description'])

percent_missing = df.isnull().sum() * 100 / len(df)
print(round(percent_missing,2))

title             0.0
authors           0.0
categories        0.0
description       0.0
average_rating    0.0
ratings_count     0.0
dtype: float64


In [ ]:
# df.drop(df[df['num_pages'] == 0].index, inplace = True)
# df['num_pages'].min()

**MODEL BUILDING-->**

In [6]:
##Convert columns values in lowercase
df['description']=df['description'].str.lower()
df['title']=df['title'].str.lower()
df['authors']=df['authors'].str.lower()
df['categories']=df['categories'].str.lower()

Combine book and rating using .merge

In [ ]:
# Index dataset using Title
#df = df.set_index(['isbn13']) #Indexed dataset


In [7]:
df.head()

,title,authors,categories,description,average_rating,ratings_count
0,gilead,marilynne robinson,fiction,a novel that readers and critics have been eag...,3.85,361.0
1,spider's web,charles osborne;agatha christie,detective and mystery stories,a new 'christie for christmas' -- a full-lengt...,3.83,5164.0
2,the one tree,stephen r. donaldson,american fiction,volume two of stephen donaldson's acclaimed se...,3.97,172.0
3,rage of angels,sidney sheldon,fiction,"a memorable, mesmerizing heroine jennifer -- b...",3.93,29532.0
4,the four loves,clive staples lewis,christian life,lewis' work on the nature of love divides love...,4.15,33684.0


**DATA WRANGLING:**
Also called Data munging, it is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics

Order of execution: Convert to lowercase, remove stop words, then lemmatize

In [8]:
import nltk
nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('omw-1.4')

#from collections import Counter 

#from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vismaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vismaya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vismaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vismaya\AppData\Roaming\nltk_data...


In [9]:
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
import re

In [10]:
import string
#REMOVE PUNCTUATIONS
def rem_punctuations(text):
    for punctuation in string.punctuation: #string.punctuation gives all set of punctuations
        text = text.replace(punctuation, '')
    return text

df['description'] = df['description'].apply(rem_punctuations)

# df['authors']=df['authors'].apply(rem_punctuations)
# df['categories']=df['categories'].apply(rem_punctuations)

In [11]:
# REMOVE NON ASCII CHARACTERS
def remove_non_ascii(string):
    return "".join(c for c in string if ord(c) < 128)

df['description'] = df['description'].apply(remove_non_ascii)
# df['title']=df['title'].apply(remove_non_ascii)
# df['authors']=df['authors'].apply(remove_non_ascii)
# df['categories']=df['categories'].apply(remove_non_ascii)

# REMOVE HTML CODES
def rem_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df['description'] = df['description'].apply(rem_html)
# df['authors']=df['authors'].apply(rem_html)
# df['categories']=df['categories'].apply(rem_html)

In [12]:
#remove stopwords
from nltk.corpus import stopwords

def rem_stopwords(text):
    return " ".join(word for word in str(text).split() if word not in set(stopwords.words('english')))

df['description'] = df['description'].apply(rem_stopwords)
# df['categories']=df['categories'].apply(rem_stopwords)

**Lemmatization of Words**

In [13]:
wln = WordNetLemmatizer()

def lem_words(text):
    return " ".join(wln.lemmatize(word) for word in text.split())

df['description']=df['description'].apply(lem_words)
#df['authors']=df['authors'].apply(lem_words)
#df['categories']=df['categories'].apply(lem_words)

**Remove Extra Spaces**

In [14]:
df['description']=df['description'].str.strip()
# df['authors']=df['authors'].str.strip()
# df['categories']=df['categories'].str.strip()

In [ ]:
#Joining Authors name with space
# def join_space(text):
#     return text.replace(" ","")

# df['authors']=df['authors'].apply(join_space)

In [15]:
## WEighted average
v=df['ratings_count'] #11800, 4500, 4466, etc.
R=df['average_rating'] # 7.2, 6.9, 6.3, 7.6, etc.
C=df['average_rating'].mean() # 6.092171559442011
m=df['average_rating'].quantile(0.70) # 581.0

df['weighted_rating']=((R*v)+ (C*m))/(v+m)

df['weighted_rating']

0       3.850915
1       3.830080
2       3.969109
3       3.930000
4       4.149973
          ...   
6803    3.733730
6804    3.820290
6805    4.488116
6808    3.931657
6809    3.767245
Name: weighted_rating, Length: 6446, dtype: float64

In [16]:
df.head()

,title,authors,categories,description,average_rating,ratings_count,weighted_rating
0,gilead,marilynne robinson,fiction,novel reader critic eagerly anticipating decad...,3.85,361.0,3.850915
1,spider's web,charles osborne;agatha christie,detective and mystery stories,new christie christmas fulllength novel adapte...,3.83,5164.0,3.830080
2,the one tree,stephen r. donaldson,american fiction,volume two stephen donaldsons acclaimed second...,3.97,172.0,3.969109
3,rage of angels,sidney sheldon,fiction,memorable mesmerizing heroine jennifer brillia...,3.93,29532.0,3.930000
4,the four loves,clive staples lewis,christian life,lewis work nature love divide love four catego...,4.15,33684.0,4.149973


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string
def tostr(text):
    return ''.join(text)

df['description']=df['description'].apply(tostr)
print(df['description'][0])
type(df['description'][0])

novel reader critic eagerly anticipating decade gilead astonishingly imagined story remarkable life john ames preacher son preacher grandson maternal paternal preacher 1956 gilead iowa towards end reverend amess life absorbed recording family story legacy young son never see grow haunted grandfather presence john tell rift grandfather father elder angry visionary fought abolitionist cause son ardent pacifist troubled prodigal namesake jack john ames boughton best friend lost son return gilead searching forgiveness redemption told john amess joyous rambling voice find beauty humour truth smallest life detail gilead song celebration acceptance best worst world offer heart tale sacred bond father son pitchperfect style story set dazzle critic reader alike


str

In [18]:
df['keyword']=df['description']+" "+df['authors']


In [36]:
#fetch category inside function- working recommend function

def recommend(title):
    title= title.lower()
    
    #Fetch the category of our title
    titlerow = df.loc[df['title']== title].iloc[0]
    category=titlerow['categories']

    # MATCH THE CATEGORY WITH THE COLUMN "CATEGORIES" OF THE DATASET
    data = df.loc[df['categories'] == category].copy()

    if len(data)<=10: ##As our dataset is unbalanced, if the matching category contains less than 10 book titles
      data=df.copy()          #,then we ommit the category filtering


    # RESET INDEX
    data.reset_index(level = 0, inplace = True, drop=True) 
    
    # INDEX TO A PANDAS SERIES
    indices = pd.Series(data.index, index = data['title'])
    
    # CONVERT THE BOOK TITLE INTO VECTORS AND USE BIGRAM
    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english',sublinear_tf=True)
    
    # tfidf_matrix = tf.fit_transform(data['title'])
    #sublinear_tfbool, default=False
    #Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf)
    
    tfidf_matrix = tf.fit_transform(data['keyword'])
    
    # CALCULATE THE SIMILARITY MEASURE
    similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # GET THE INDEX OF ORIGINAL TITLE
    title_index = indices[title].tolist()
    print("title_index",title_index)
    if not(type(title_index) is int): 
        
        title_index=title_index[0]    #if more than one matching index exists, take the 1st one
        inds=indices[title].tolist()
        for i in inds:   #to drop other rows with the same title
          if i!=title_index:
            print('index dropped:',i)
            data.drop(i,inplace=True)
    
    # PAIRWISE SIMILARITY SCORES
    similarity = list(enumerate(similarity[title_index]))
    
    # SORT THE BOOKS
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)

    # GET TOP 10 MOST SIMILAR BOOKS
    similarity  = similarity [1:11]
    print("similarity:",similarity,"\n")
    book_indices = [i[0] for i in similarity]

    #Weighted Rating method
    top10_rated = data['weighted_rating'].iloc[book_indices]
    wsort = top10_rated.sort_values(ascending = False)
    wsort_top5 = wsort[:6]
    wsort_top5.to_frame()

    # INDICES OF TOP 5
    wsort_indices = wsort_top5.index
    #print(wsort_indices)

    # TOP 5 RECOMMENDATION
    rec = data[['title']].iloc[wsort_indices]
    
    # PRINT THE BOOKS TITLE
    print("\n","The recommendations are:")
    print(rec['title'])

In [37]:
recommend("Murder on the Orient Express")

title_index [1110, 2382]
index dropped: 2382
similarity: [(2387, 0.08564157267058235), (2389, 0.0778467844233684), (2382, 0.07584790701103701), (368, 0.07337464099435693), (2390, 0.06578136556475976), (606, 0.0549890514894677), (2386, 0.050347001119664156), (2383, 0.05029005291203832), (519, 0.04764768220636827), (3, 0.04191084589035031)] 


 The recommendations are:
519     absent in the spring and other novels
2385                  the body in the library
2392                                  prophet
2384                   murder at the vicarage
2389                        death on the nile
2388                    a murder is announced
Name: title, dtype: object


In [38]:
recommend("harry potter")

title_index 231
similarity: [(243, 0.8448622967175232), (218, 0.08208474106560615), (193, 0.0491516362182596), (202, 0.04817069815050505), (219, 0.04169553075617062), (106, 0.04113347348944385), (291, 0.04064722627952388), (239, 0.027965698946575898), (390, 0.02265937358058459), (229, 0.012557868183357366)] 


 The recommendations are:
243                          the harry potter collection
239      harry potter and the half-blood prince (book 6)
229    harry potter and the prisoner of azkaban (book 3)
390                  harry potter and the goblet of fire
202    harry potter and the order of the phoenix (boo...
219       harry potter and the sorcerer's stone (book 1)
Name: title, dtype: object


In [39]:
recommend('a brief History of time')

title_index 3266
similarity: [(3269, 0.15777878240489793), (2325, 0.05440497467778775), (2278, 0.04713636249146537), (3535, 0.046677060244957866), (5974, 0.04355468135128589), (5975, 0.04355468135128589), (5558, 0.0422856323481561), (1231, 0.03939145669247465), (4325, 0.038132906759446014), (4909, 0.038132906759446014)] 


 The recommendations are:
3269    the illustrated a brief history of time
3535    the illustrated a brief history of time
2278              the annotated christmas carol
1231          contemporary political philosophy
4909           twelfth night, or, what you will
2325                    the future of spacetime
Name: title, dtype: object


In [40]:
# TEST
#recommend("death note")
recommend("Mahatma Gandhi and His Myths")

title_index 26
similarity: [(0, 0.012193128473190555), (4, 0.006878282708655834), (1, 0.0), (2, 0.0), (3, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0)] 


 The recommendations are:
0       declarations of independence
1                       the crusader
5                seeing like a state
2                 homegrown democrat
9    the collapse of the common good
4     the intellectuals and the flag
Name: title, dtype: object


In [41]:
recommend("agile web development with rails")

title_index [30, 32]
index dropped: 32
similarity: [(32, 0.15548867862586885), (16, 0.01805570349835141), (33, 0.01494501132498283), (26, 0.014021036860196262), (29, 0.010683190192862096), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)] 


 The recommendations are:
16                                 hackers & painters
29                                   programming ruby
2                                                java
4                  robin williams web design workshop
0     incident response & computer forensics, 2nd ed.
26                          php and mysql for dummies
Name: title, dtype: object


In [42]:
recommend("the da vinci code")

title_index [986, 2265]
index dropped: 2265
similarity: [(2265, 0.23272001551643492), (2315, 0.04941560732574825), (418, 0.03283304485458853), (1341, 0.024515584006773512), (288, 0.022477165089281284), (2316, 0.019433503015860887), (285, 0.01877549695808719), (1346, 0.017049767167448472), (1917, 0.016906604689856046), (1769, 0.016566195718722962)] 


 The recommendations are:
1341    les miserables a new unabridged translation
288                                  les miserables
2267                                        company
418                            flowers for algernon
1769                             the tombs of atuan
1346                    the hunchback of notre-dame
Name: title, dtype: object
